In [1]:
import matplotlib
import pandas as pd
import numpy as np
import time
from datetime import datetime
from re import sub
def mdy_to_ymd(d):
    return time.mktime(datetime.strptime(d, '%d-%b-%y').timetuple())

class Beautify:

    @staticmethod
    def beautify(path):
        dataset = pd.read_csv(path)
        dataset.columns = ["date", "price", "open", "high", "low", "vol", "change"]
        dataset["date"] = dataset.apply(lambda row: mdy_to_ymd(row['date']), axis=1)
        for col in ["price", "open", "high", "low", "vol", "change"]:
            dataset[col] = dataset.apply(
                lambda row: (lambda x: 0 if x == '-' else float(x))(sub(r'[^\d.-]', '', row[col])),
                axis=1)
        dataset = dataset.sort_values(["date"])
        return dataset


pd.set_option('display.float_format', lambda x: '%.0f' % x)

In [2]:
dataset = Beautify.beautify("dataset.csv")[["date","price"]]

In [3]:
TRAIN = time.mktime(datetime.strptime("20-01-02", '%y-%m-%d').timetuple())
train = dataset[dataset.date < TRAIN]
test = dataset[dataset.date >= TRAIN]

In [4]:
train.columns = ["ds","y"]
test.columns = ["ds","y"]

In [ ]:
train["ds"] = train.apply(lambda x: datetime.utcfromtimestamp(x["ds"]).strftime('%Y-%m-%d'),axis=1)
test["ds"] = test.apply(lambda x: datetime.utcfromtimestamp(x["ds"]).strftime('%Y-%m-%d'),axis=1)
from fbprophet import Prophet
prophet = Prophet(interval_width = 0.95,
yearly_seasonality = True,
weekly_seasonality = True,
daily_seasonality = True,
holidays = None,
changepoint_prior_scale = 0.05)
prophet.fit(train)
future = prophet.make_future_dataframe(priods=500)
prophet.predict(future)